In [1]:
import tensorflow as tf
from tensorflow import keras 

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
import numpy as np
from keras.utils import to_categorical


# Create a data generator for data augmentation
datagen = ImageDataGenerator(
        rotation_range=15,    # rotate the image by up to 15 degrees
        width_shift_range=0.1,    # shift the image horizontally by up to 10%
        height_shift_range=0.1,   # shift the image vertically by up to 10%
        shear_range=0.1,     # apply shear transformation by up to 10%
        zoom_range=0.1,      # zoom in or out on the image by up to 10%
        horizontal_flip=True,    # flip the image horizontally
        fill_mode='nearest')  # fill any missing pixels with the nearest value

# Fit the data generator to the training data
datagen.fit(x_train)

# Generate augmented data
augmented_data = datagen.flow(x_train, y_train, batch_size=32)

# Concatenate the original training data with the augmented data
x_train_augmented = np.concatenate((x_train, augmented_data[0][0]), axis=0)
y_train_augmented = np.concatenate((y_train, augmented_data[0][1]), axis=0)

# Convert the labels to one-hot vectors
num_classes = 10
y_train_augmented = to_categorical(y_train_augmented, num_classes)
y_test = to_categorical(y_test, num_classes)

In [5]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping


# Load the pre-trained VGG16 model
vgg16 = VGG16(weights='imagenet',
              include_top=False,
              input_shape=(32, 32, 3))

# Freeze the weights of the pre-trained layers
for layer in vgg16.layers:
    layer.trainable = False

# Add new fully connected layers on top of the pre-trained layers
x = vgg16.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# Create a new model with the new fully connected layers
model = Model(inputs=vgg16.input, outputs=predictions)

# Compile the model
model.compile(optimizer=RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

batch_size = 32
epochs = 50

# Train the model
history = model.fit(x_train_augmented, y_train_augmented,
                    batch_size=batch_size,
                    epochs=50,
                    #steps_per_epoch=len(x_train) / 32,
                    validation_data=(x_test, y_test),
                     callbacks=[early_stop])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
1564/1564 [==============================] - 23s 13ms/step - loss: 7.1537 - accuracy: 0.2143 - val_loss: 2.1417 - val_accuracy: 0.2651
Epoch 2/50
1564/1564 [==============================] - 17s 11ms/step - loss: 2.2496 - accuracy: 0.2755 - val_loss: 1.8505 - val_accuracy: 0.3823
Epoch 3/50
1564/1564 [==============================] - 18s 11ms/step - loss: 2.0332 - accuracy: 0.3427 - val_loss: 1.6865 - val_accuracy: 0.4319
Epoch 4/50
1564/1564 [==============================] - 17s 11ms/step - loss: 1.9110 - accuracy: 0.3762 - val_loss: 1.5847 - val_accuracy: 0.4564
Epoch 5/50
1564/1564 [==============================] - 17s 11ms/step - loss: 1.8333 - accuracy: 0.4016 - val_loss: 1.5276 - val_accuracy: 0.4749
Epoch 6/50
1564/1564 [==============================] - 17s 11ms/step - loss: 1.7762 - accuracy: 0.4200 - val_loss: 1.4662 - val_accuracy: 0.5006
Epoch 7/50
1564/1564 [==============================] - 17s 11ms/step - loss: 1.7375 - accuracy: 0.4356 - val_loss: 1.4504 -

In [6]:
# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.3898941278457642
Test accuracy: 0.5893999934196472
